# Cuaderno de Documentación: 05 - Utilidades de Reporte (`report_utils.py`)

El módulo `scripts/report_utils.py` se encarga de la etapa final del procesamiento: tomar la respuesta cruda generada por el LLM, interpretarla, estructurarla y guardarla en un formato legible y útil. Esto implica parsear el JSON esperado de la respuesta del LLM y asignar una clasificación de riesgo visual (Rojo, Ámbar, Verde).

## Funcionalidades Principales

1.  **`intentar_parsear_json_riesgos(texto_llm_str)`:**
    * **Propósito:** Extraer de forma robusta un objeto JSON de la cadena de texto devuelta por el LLM. A menudo, los LLMs pueden envolver su salida JSON en bloques de código Markdown (ej. ` ```json ... ```) o añadir texto introductorio/conclusivo.
    * **Proceso:**
        * Verifica que la entrada sea un string.
        * Limpia espacios en blanco al inicio y al final del texto.
        * Intenta detectar si la respuesta está envuelta en bloques de código Markdown ` ```json ... ``` o ` ``` ... ``` y extrae el contenido JSON interno.
        * Si no está en un bloque de código, busca el primer `{` y el último `}` para delimitar el JSON, lo que puede ayudar si hay texto adicional fuera del JSON principal.
        * Utiliza `json.loads()` para convertir la cadena candidata a JSON en un objeto Python (diccionario/lista).
    * **Retorno:** Un diccionario o lista Python si el parseo es exitoso, o `None` si no se puede decodificar un JSON válido.

2.  **`asignar_estado_rag(impacto_str, probabilidad_str)`:**
    * **Propósito:** Asignar un color de semáforo (Rojo, Ámbar, Verde) o un estado "Gris (Indeterminado)" a un riesgo, basándose en las cadenas de texto para impacto y probabilidad que se espera que el LLM proporcione (ej. "Alto", "Medio", "Baja").
    * **Proceso:**
        * Convierte las cadenas de impacto y probabilidad a minúsculas y elimina espacios para una comparación consistente.
        * Aplica una lógica de matriz de riesgo simplificada:
            * **Rojo (Alto Riesgo):** Generalmente para combinaciones de Alto Impacto con Media/Alta Probabilidad, o Medio Impacto con Alta Probabilidad.
            * **Ámbar (Riesgo Moderado):** Para combinaciones intermedias.
            * **Verde (Bajo Riesgo):** Generalmente para Bajo Impacto con Baja/Media Probabilidad.
            * **Gris (Indeterminado):** Si el impacto o la probabilidad son desconocidos o no coinciden con las categorías esperadas.
    * **Retorno:** Una cadena de texto que representa el estado RAG del riesgo.

3.  **`formatear_y_guardar_reporte(resultado_analisis_llm, fuentes_recuperadas, nombre_pdf_proyecto, modelo_llm_usado, output_path_dir)`:**
    * **Propósito:** Orquestar el procesamiento de la salida del LLM y guardar el reporte final estructurado.
    * **Proceso:**
        * Inicializa una estructura de diccionario para el `reporte_final`, incluyendo metadatos como timestamp, nombre del proyecto analizado, modelo LLM usado, y la respuesta cruda del LLM.
        * Procesa la lista de `fuentes_recuperadas` (que ya debería ser una lista de diccionarios serializables pasados por el orquestador) y las añade al reporte.
        * Llama a `intentar_parsear_json_riesgos()` para obtener los riesgos estructurados de `resultado_analisis_llm`.
        * Si el parseo es exitoso y se encuentra la clave `"riesgos_identificados"` como una lista:
            * Itera sobre cada ítem de riesgo.
            * Extrae los campos esperados (`descripcion_riesgo`, `explicacion_riesgo`, `impacto_estimado`, `probabilidad_estimada`).
            * Llama a `asignar_estado_rag()` para obtener el estado RAG sugerido.
            * Añade el riesgo procesado (con su estado RAG) a la lista `"riesgos_identificados_estructurados"` del reporte final.
        * Genera un nombre de archivo único para el reporte JSON, incluyendo el nombre del proyecto y un timestamp.
        * Guarda el `reporte_final` como un archivo JSON en el directorio `output_path_dir` (normalmente `datos/Resultados/`), utilizando `json.dump()` con indentación para facilitar la lectura.

A continuación, se muestra el contenido del script `report_utils.py` para referencia.

In [ ]:
# Cargar el contenido del script report_utils.py
%load ../scripts/report_utils.py